자체 채점 포함 버전 (testset 데이터 활용)

In [ ]:
!git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

Cloning into 'LightGBM'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 20846 (delta 6), reused 6 (delta 5), pack-reused 20827
Receiving objects: 100% (20846/20846), 16.58 MiB | 12.18 MiB/s, done.
Resolving deltas: 100% (15211/15211), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728), 8

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame, concat
from google.colab import drive
drive.mount('/content/drive')
from datetime import date
import lightgbm as lgbm
from lightgbm import LGBMRegressor
import time
from sklearn.model_selection import train_test_split
import os

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=388001b87276aa42d57ba4d325505bff89cdb19af81de9f9eca2c5842f6bd6c5
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
Mounted at /content/drive


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import math
from math import radians
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#### 파라미터 ####

# 하루의 틱
ticks = 48

# 예측에 사용할 일수
days = 3                # 이 부분을 바꿀 수 있다.
n_days = ticks*days

# 미래 예측할 일수
future_days = 2
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']

# 사용할 변수
what_to_left = ['Time', 'WS', 'DHI','DNI','RH','T','TARGET', 'Day']

n_features = len(what_to_left)
n_obs = n_days * n_features # 7일 예측 기준으로 7(일 수) * 48(틱 수) * 6(변수 개수)

In [ ]:
# 데이터 불러오기 

raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/Dacon/unlimited_power/raw_data/train/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/Jupyter/Dacon/unlimited_power/raw_data/sample_submission.csv')
submission.shape

(7776, 10)

In [ ]:
def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset['Date'] = dataset['Day']%365
  dataset.drop('Day', axis=1, inplace=True)
  
  temp = list()
  for i in range(0, len(dataset), 48):
    temp += [24-(list(dataset.DHI[i:i+48]).count(0)*0.5)]*48 

  dataset["SH"] = temp

  return dataset

def cos_time(dataset):
  dataset['sin_time'] = np.sin(2*np.pi*dataset.Time/24)
  dataset['cos_time'] = np.cos(2*np.pi*dataset.Time/24)
  return dataset

In [ ]:
def get_yday(when):
  HP = date(2020,1,1)
  results = (when-HP).days
  return results

In [ ]:
def 절기24(Date):
  target = 0
  for i in list24:
    if Date < i:
      target = list24.index(i) - 1
      break
  if Date < 5:
    target = 23
  return target

In [ ]:
def train_to_supervised(train, target, n_in):

    clmns = list(train.columns)

    # 기타 칼럼은 전과 같이 들어갈 것.
    scaled_lst = clmns

    scaled_df = train[scaled_lst]
    target_df = target

    # 미래 몇 번째 항목을 가져올 것인가
    future = [48, 96]

    ### 만약에 스케일링을 하고 싶다면 ###
    # scaled_df 데이터 프레임만 스케일링 하고, 절기랑 TARGET 데이터는 그냥 두면 된다.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    n_vars2 = 1 if type(target_df) is list else target_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # 48과 96 후의 타겟 데이터 2개 붙이기.
    # forecast sequence (t, t+1, ... t+n)
    for i in future:
        cols.append(target_df.shift(-i))
        if i == 0:
            names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
        else:
            names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [ ]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def test_to_supervised(train, n_in):

    clmns = list(train.columns)

    # 타켓 칼럼의 이름을 여기에 입력
    target = ['TARGET']

    # 클래스 변수로 전환(encoding)할 칼럼을 여기에 입력
#    class_lst = ['season']

    # 기타 칼럼은 전과 같이 들어갈 것.
#    scaled_lst = list(set(clmns) - set(class_lst))

    scaled_df = train.copy()
#    class_df = train[class_lst]

    ### 만약에 스케일링을 하고 싶다면 ###
    # testset의 스케일링은 구조가 상당히 까다로우므로....... 일단 나중에 하기로 함.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # class라 encoding 필요한 절기 끝에 하나만 붙여놓기
#    cols.append(class_df)
#    names += class_lst
    
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [ ]:
소한 = get_yday(date(2020,1,6))
대한 = get_yday(date(2020,1,20))
입춘 = get_yday(date(2020,2,4))
우수 = get_yday(date(2020,2,19))
경칩 = get_yday(date(2020,3,6))
춘분 = get_yday(date(2020,3,21))
청명 = get_yday(date(2020,4,5))
곡우 = get_yday(date(2020,4,20))
입하 = get_yday(date(2020,5,6))
소만 = get_yday(date(2020,5,21))
망종 = get_yday(date(2020,6,6))
하지 = get_yday(date(2020,6,22))
소서 = get_yday(date(2020,7,7))
대서 = get_yday(date(2020,7,23))
입추 = get_yday(date(2020,8,8))
처서 = get_yday(date(2020,8,23))
백로 = get_yday(date(2020,9,8))
추분 = get_yday(date(2020,9,23))
한로 = get_yday(date(2020,10,8))
상강 = get_yday(date(2020,10,24))
입동 = get_yday(date(2020,11,8))
소설 = get_yday(date(2020,11,22))
대설 = get_yday(date(2020,12,7))
동지 = get_yday(date(2020,12,22))

list24 = [소한, 대한, 입춘, 우수, 경칩, 춘분, 청명, 곡우, 입하, 소만, 망종, 하지, 소서, 대서, 입추, 처서, 백로, 추분, 한로, 상강, 입동, 소설, 대설, 동지]

In [ ]:
m = 60

일출 = [7+47/m, 7+44/m, 7+34/m, 7+18/m, 6+56/m, 6+35/m, 6+12/m, 5+51/m, 5+33/m, 5+19/m, 5+11/m, 5+11/m, 5+17/m, 5+27/m, 5+41/m, 5+54/m, 6+8/m, 6+20/m, 6+33/m, 6+47/m, 7+3/m, 7+20/m, 7+33/m, 7+45/m]
일몰 = [17+28/m, 17+42/m, 17+58/m, 18+15/m, 18+30/m, 18+44/m, 18+58/m, 19+12/m, 19+25/m, 19+39/m, 19+50/m, 19+57/m, 19+56/m, 19+50/m, 19+35/m, 19+15/m, 18+51/m, 18+28/m, 18+6/m, 17+45/m, 17+28/m, 17+17/m, 17+13/m, 17+17/m]
경사각 = [32.92, 36.83, 40.75, 44.67, 48.58, 52.5, 56.42, 60.33, 64.25, 68.16, 72.01, 76, 72.1, 68.16, 64.25, 60.33, 56.42, 52.5, 48.58, 44.67, 40.75, 36.83, 32.92, 29]
남중 = [12+36/m, 12+41/m, 12+44/m, 12+44/m, 12+42/m, 12+38/m, 12+34/m, 12+29/m, 12+27/m, 12+27/m, 12+29/m, 12+32/m, 12+35/m, 12+37/m, 12+36/m, 12+33/m, 12+29/m, 12+23/m, 12+18/m, 12+15/m, 12+14/m, 12+16/m, 12+22/m, 12+29/m]
print(len(일출), len(일몰), len(경사각), len(남중))

24 24 24 24


In [ ]:
def HRA(DHI, DNI, season, hour):
  # 위도(latitude) 기준을 일단 임의로 대전으로 설정 (위도 36.19~36.2도)
  latitude = radians(36.2)
  season = int(season)
  # 절기별 대한민국의 경사각
  tilt = radians(경사각[season])

  # 절기별 대한민국 대전의 태양 남중시각
  hra = radians(15*(hour - 남중[season]))

  # 구하려는 알파
  elevation = np.arcsin(np.sin(tilt) * np.sin(latitude) + np.cos(tilt) * np.cos(latitude) * np.cos(hra))

  # 천정각(Zenith Angle)은 90 - 알파
  zenith = radians(90) - elevation

  # GHI는 DHI + DNI * cos(천정각)
  ghi = DHI + DNI *np.cos(zenith)

  return ghi

In [ ]:
def encoding(data):
  season_list = [i for i in range(0,24)]
  for k in season_list:
    data['Season_' +f'{k}'] = data['season'] == k
  return data*1

In [ ]:
def not_minus(dataset):
  for i in range(0, len(dataset.index)):
    for j in range(0, len(dataset.columns)):
      K = dataset.iloc[i,j]
      if K < 0:
        dataset.iloc[i,j] = 0
  return dataset

In [ ]:
dataset = raw_data.copy()
dataset2 = drop_clms(dataset)
dataset2.drop(['Hour', 'Minute'], axis =1, inplace=True)

In [ ]:
dataset2['season'] = dataset2.apply(lambda x: 절기24(x['Date']), axis = 1)
dataset3 = cos_time(dataset2)
dataset4 = dataset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
# dataset5 = dataset4.iloc[n_days:,:-1]
# goals = pd.DataFrame(dataset4.Goal, index = dataset4.index)

In [ ]:
dataset5= dataset4.copy()

dataset5['GHI'] = dataset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

##### 변수 바꾸기

# 만약에 변수를 바꿔서 넣어보고 싶다면 밑에 
##dataset6 = dataset5[[리스트]] 에서 리스트 안에 변수를 수정

In [ ]:
#dataset6 = dataset5[['DHI', 'DNI', 'WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'season', 'TARGET']]

##################### 무슨 변수를 넣어보고 싶은지 #############################
variables = ['RH', 'T', 'SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']
dataset6 = dataset5[variables]

In [ ]:
scaler = MinMaxScaler()
# scaler = StandardScaler()
scaler.fit(dataset6)
temp_X = pd.DataFrame(scaler.transform(dataset6), columns = dataset6.columns)
temp_y = pd.DataFrame(dataset6['TARGET'])

In [ ]:
dataset7 = train_to_supervised(temp_X,temp_y, n_days)

In [ ]:
X = dataset7.iloc[:,:-2]
#X = encoding(X)
#X.drop('season', axis =1, inplace=True)

y_1 = pd.DataFrame(dataset7.iloc[:,-2])
y_2 = pd.DataFrame(dataset7.iloc[:,-1])

In [ ]:
X_train_1, X_valid_1, Y_train_1,  Y_valid_1 = train_test_split(X, y_1, test_size=0.30, random_state=42)
X_train_2, X_valid_2, Y_train_2,  Y_valid_2 = train_test_split(X, y_2, test_size=0.30, random_state=42)

### 공개된 TARGET 데이터는 하나 뿐이라 quantile 0.5 기준으로 채점 함.
k=0
quantiles = 0.5

### 파라미터 바꾸기

## 요기에 원하는 파라미터를 넣고 돌려본다

In [ ]:
################## 여기에 확인하려는 파라미터를 넣어본다 #######################

params = {}
params['objective'] = 'quantile'
params['min_child_samples'] = 10
params['min_child_weight'] = 150
params['min_split_gain'] = 0
params['n_estimators'] = 2000
params['bagging_fraction'] = 0.9
params['feature_fraction'] = 0.9
params['learning_rate'] = 0.007
params['save_binary'] = True
params['is_unbalance'] = False
params['subsample'] = 0.9
params['max_drop'] = 50
params['max_bin'] = 63
params['num_leaves'] = 40
params['device'] = 'gpu'
params['max_depth'] = 25
params['boosting_type'] = "gbdt"

In [ ]:
# LGBM 모델을 구축하고 예측까지 하는 함수를 만든다. 
#def LGBM(q, X, Y, X_test):

def LGBM(q, X_train, X_valid, Y_train, Y_valid):    
    # (a) 모델링
    model = LGBMRegressor(**params, alpha=q)          

    model.fit(X_train, Y_train, eval_metric = ['quantile'], eval_set=[(X_valid, Y_valid)], early_stopping_rounds=500, verbose=500)

    return model

In [ ]:
tick = time.time()

# Target1
models_1 = LGBM(quantiles, X_train_1, X_valid_1, Y_train_1, Y_valid_1)
# Target2
models_2 = LGBM(quantiles, X_train_2, X_valid_2, Y_train_2,  Y_valid_2)

tock = time.time()
print('소요시간 : ', (tock - tick)/60, '분')

Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 2.71324
[1000]	valid_0's quantile: 2.51472
[1500]	valid_0's quantile: 2.44435
[2000]	valid_0's quantile: 2.40695
Did not meet early stopping. Best iteration is:
[2000]	valid_0's quantile: 2.40695
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 2.74052
[1000]	valid_0's quantile: 2.53282
[1500]	valid_0's quantile: 2.47343
[2000]	valid_0's quantile: 2.43843
Did not meet early stopping. Best iteration is:
[2000]	valid_0's quantile: 2.43843
소요시간 :  1.352300723393758 분


# 여기서부터 채점
## Testset에서 훔쳐온 7700여개의 데이터를 predict에 넣어서
## 실제 값이랑 predict 예측값이랑 얼마나 차이가 나는가 개략적으로 볼 수 있음.
## 결과 값은 맨 아래에 나오고... 이쪽은 크게 건드릴 필요는 없을 듯?

In [ ]:
df_test = []

# 예시 코드
# 아직 유진이 어떻게 변수 추가할 지 몰라서 to_supervised는 쓰지 않았음.

for i in range(81):
    file_path = '/content/drive/MyDrive/Jupyter/Dacon/unlimited_power/raw_data/test/' + str(i) + '.csv'
    new_path = '/content/drive/MyDrive/Jupyter/Dacon/unlimited_power/raw_data/test/adj_test/' + str(i) + '.csv'
    file_name = str(i) + '.csv'
    
    # adj_test에서 가져와야 하므로 new_path에서 불러온다
    temp = pd.read_csv(new_path)
    temp.drop('Unnamed: 0', axis = 1, inplace = True)
    testset = temp.copy()

    # 24로 저장한 24절기 season으로 이름 바꾸기
    testset.rename(columns = {'24' : 'season'}, inplace = True)

    # 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
    testset2 = drop_clms(testset)
    testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
    testset3 = cos_time(testset2)
    testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]

    testset5 = testset4.copy()
    testset5['GHI'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)
    testset6 = testset5[variables] #'DHI', 'DNI', 'season', 'WS', 'SH'
    temp_X = pd.DataFrame(scaler.transform(testset6), columns = testset6.columns)
    temp_y = pd.DataFrame(testset6['TARGET'])
    testset7 = train_to_supervised(temp_X, temp_y, n_days)

#    testset8 = encoding(testset7)
#    testset8.drop('season', axis = 1, inplace = True)
    # testset9 = testset7.iloc[-48:,:]

    df_test.append(testset7)

X_test = pd.concat(df_test)
# X_test = X_test.iloc[:, :n_obs]
X_test 

,RH(t-144),T(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHI(t-144),TARGET(t-144),RH(t-143),T(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHI(t-143),TARGET(t-143),RH(t-142),T(t-142),SH(t-142),sin_time(t-142),cos_time(t-142),GHI(t-142),TARGET(t-142),RH(t-141),T(t-141),SH(t-141),sin_time(t-141),cos_time(t-141),GHI(t-141),TARGET(t-141),RH(t-140),T(t-140),SH(t-140),sin_time(t-140),cos_time(t-140),GHI(t-140),TARGET(t-140),RH(t-139),T(t-139),SH(t-139),sin_time(t-139),cos_time(t-139),...,cos_time(t-6),GHI(t-6),TARGET(t-6),RH(t-5),T(t-5),SH(t-5),sin_time(t-5),cos_time(t-5),GHI(t-5),TARGET(t-5),RH(t-4),T(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),RH(t-3),T(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),RH(t-2),T(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),RH(t-1),T(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1),TARGET1(t+48),TARGET1(t+96)
144,0.290337,0.351852,0.166667,0.500000,1.000000,0.0,0.0,0.287631,0.353704,0.166667,0.565263,0.995722,0.0,0.0,0.288280,0.355556,0.166667,0.629410,0.982963,0.0,0.0,0.285683,0.357407,0.166667,0.691342,0.961940,0.0,0.0,0.285467,0.359259,0.166667,0.750000,0.933013,0.0,0.0,0.280164,0.362963,0.166667,0.804381,0.896677,...,0.853553,0.000000,0.000000,0.490315,0.357407,0.166667,0.195619,0.896677,0.000000,0.000000,0.508062,0.359259,0.166667,0.250000,0.933013,0.0,0.0,0.503084,0.361111,0.166667,0.308658,0.961940,0.0,0.0,0.522238,0.361111,0.166667,0.370590,0.982963,0.0,0.0,0.530246,0.357407,0.166667,0.434737,0.995722,0.0,0.0,0.0,0.0
145,0.287631,0.353704,0.166667,0.565263,0.995722,0.0,0.0,0.288280,0.355556,0.166667,0.629410,0.982963,0.0,0.0,0.285683,0.357407,0.166667,0.691342,0.961940,0.0,0.0,0.285467,0.359259,0.166667,0.750000,0.933013,0.0,0.0,0.280164,0.362963,0.166667,0.804381,0.896677,0.0,0.0,0.285467,0.364815,0.166667,0.853553,0.853553,...,0.896677,0.000000,0.000000,0.508062,0.359259,0.166667,0.250000,0.933013,0.000000,0.000000,0.503084,0.361111,0.166667,0.308658,0.961940,0.0,0.0,0.522238,0.361111,0.166667,0.370590,0.982963,0.0,0.0,0.530246,0.357407,0.166667,0.434737,0.995722,0.0,0.0,0.557299,0.353704,0.166667,0.500000,1.000000,0.0,0.0,0.0,0.0
146,0.288280,0.355556,0.166667,0.629410,0.982963,0.0,0.0,0.285683,0.357407,0.166667,0.691342,0.961940,0.0,0.0,0.285467,0.359259,0.166667,0.750000,0.933013,0.0,0.0,0.280164,0.362963,0.166667,0.804381,0.896677,0.0,0.0,0.285467,0.364815,0.166667,0.853553,0.853553,0.0,0.0,0.279732,0.368519,0.166667,0.896677,0.804381,...,0.933013,0.000000,0.000000,0.503084,0.361111,0.166667,0.308658,0.961940,0.000000,0.000000,0.522238,0.361111,0.166667,0.370590,0.982963,0.0,0.0,0.530246,0.357407,0.166667,0.434737,0.995722,0.0,0.0,0.557299,0.353704,0.166667,0.500000,1.000000,0.0,0.0,0.580132,0.344444,0.166667,0.565263,0.995722,0.0,0.0,0.0,0.0
147,0.285683,0.357407,0.166667,0.691342,0.961940,0.0,0.0,0.285467,0.359259,0.166667,0.750000,0.933013,0.0,0.0,0.280164,0.362963,0.166667,0.804381,0.896677,0.0,0.0,0.285467,0.364815,0.166667,0.853553,0.853553,0.0,0.0,0.279732,0.368519,0.166667,0.896677,0.804381,0.0,0.0,0.289795,0.372222,0.166667,0.933013,0.750000,...,0.961940,0.000000,0.000000,0.522238,0.361111,0.166667,0.370590,0.982963,0.000000,0.000000,0.530246,0.357407,0.166667,0.434737,0.995722,0.0,0.0,0.557299,0.353704,0.166667,0.500000,1.000000,0.0,0.0,0.580132,0.344444,0.166667,0.565263,0.995722,0.0,0.0,0.629477,0.337037,0.166667,0.629410,0.982963,0.0,0.0,0.0,0.0
148,0.285467,0.359259,0.166667,0.750000,0.933013,0.0,0.0,0.280164,0.362963,0.166667,0.804381,0.896677,0.0,0.0,0.285467,0.364815,0.166667,0.853553,0.853553,0.0,0.0,0.279732,0.368519,0.166667,0.896677,0.804381,0.0,0.0,0.289795,0.372222,0.166667,0.933013,0.750000,0.0,0.0,0.289795,0.372222,0.166667,0.961940,0.691342,...,0.982963,0.000000,0.000000,0.530246,0.357407,0.166667,0.434737,0.995722,0.000000,0.000000,0.557299,0.353704,0.166667,0.500000,1.000000,0.0,0.0,0.580132,0.344444,0.166667,0.565263,0.995722,0.0,0.0,0.629477,0.337037,0.166667,0.629410,0.982963,0.0,0.0,0.

In [ ]:
test_data = X_test.iloc[:,:-2]
answer_1 = pd.DataFrame(X_test.iloc[:,-2]).reset_index().drop('index', axis = 1)
answer_2 = pd.DataFrame(X_test.iloc[:,-1]).reset_index().drop('index', axis = 1)

In [ ]:
pred_1 = pd.DataFrame(pd.Series(models_1.predict(test_data).round(4)))
pred_2 = pd.DataFrame(pd.Series(models_2.predict(test_data).round(4)))

########## 점수확인

# 밑에 점수를 보면 된다.

In [ ]:
rms_1 = np.sqrt(mean_squared_error(answer_1, pred))
rms_2 = np.sqrt(mean_squared_error(answer_2, pred))

print("24시간 뒤 예측(모델1)의 점수는")
print(rms_1)
print('')
print("############" * 10)
print('')
print("48시간 뒤 예측(모델2)의 점수는")
print(rms_2)

24시간 뒤 예측(모델1)의 점수는
12.682064015512411

########################################################################################################################

48시간 뒤 예측(모델2)의 점수는
13.354669856482555
